In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import math
import glob
from IPython.display import clear_output
from pip import main

import numpy as np
import matplotlib as mpl
from matplotlib import cm
from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

In [2]:
def eulard(A,B):
    return np.sqrt(sum(np.power((A-B),2)))

# def eular3d(A,B):
#     return np.sqrt(sum(np.power((A-B),2)))

In [3]:
def show_img(img, bigger=False):
    if bigger:
        plt.figure(figsize=(15,15))
    image_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    plt.imshow(image_rgb)
    plt.show()

def mouse_handler(event, x, y, flags, data):
    if event == cv2.EVENT_LBUTTONDOWN:
        # 標記點位置
        cv2.circle(data['img'], (x,y), 3, (0,0,255), 5, 16) 

        # 改變顯示 window 的內容
        cv2.imshow("Image", data['img'])
        
        # 顯示 (x,y) 並儲存到 list中
        print("get points: (x, y) = ({}, {})".format(x, y))
        data['points'].append((x,y))

def get_points(im):
    # 建立 data dict, img:存放圖片, points:存放點
    data = {}
    data['img'] = im.copy()
    data['points'] = []
    
    # 建立一個 window
    cv2.namedWindow("Image", 0)
    
    # 改變 window 成為適當圖片大小
    h, w, dim = im.shape
    print("Img height, width: ({}, {})".format(h, w))
    cv2.resizeWindow("Image", w, h)
        
    # 顯示圖片在 window 中
    cv2.imshow('Image',im)
    
    # 利用滑鼠回傳值，資料皆保存於 data dict中
    cv2.setMouseCallback("Image", mouse_handler, data)
    
    # 等待按下任意鍵，藉由 OpenCV 內建函數釋放資源
    cv2.waitKey()
    cv2.destroyAllWindows()
    
    # 回傳點 list
    return data['points']

In [4]:
def caculation(data):
    totalPA = []
    totalPB = []
    tanglePA = []
    tanglePB = []

    for i ,element in enumerate(data):
    #     print (i,element)
    #     print(element[0])
        A = np.array([element[0][0],element[0][1]])
        B = np.array([element[1][0],element[1][1]])
        P = np.array([element[2][0],element[2][1]])
        distAB = eulard(A,B) # pixel for units
        distPA = eulard(P,A)
        distPB = eulard(P,B)
        perPA = distPA/distAB
        perPB = distPB/distAB
        anglePA = np.arccos(np.dot((B-A),(P-A))/(distAB*distPA))
        anglePB = np.arccos(np.dot((B-A),(P-B))/(distAB*distPB))

    #     print(type(perPA))
        totalPA.append(perPA)
        totalPB.append(perPB)
        tanglePA.append(anglePA)
        tanglePB.append(anglePB)



    print(totalPA)
    print(totalPB)
    avgPA = np.mean(totalPA)
    avgPB = np.mean(totalPB)
    stdPA = np.std(totalPA)
    stdPB = np.std(totalPB)
    avganglePA = np.mean(tanglePA)
    avganglePB = np.mean(tanglePB)
    stdanglePA = np.std(tanglePA)
    stdanglePB = np.std(tanglePB)
    print('avgPA,avgPB\n',avgPA,avgPB)
    print('stdPA,stdPB\n',stdPA,stdPB)
    print('stdanglePA,stdanglePB\n',stdanglePA,stdanglePB)
    
    print('the avganglePA is\n',avganglePA,"(/rad)")
    print('the avganglePA is\n',avganglePA *180/3.1415,"(/degree)")
    # print(avganglePB)
    print(tanglePA)
    # print(tanglePB)


    #     print(dist)
    #     dist = element
    
    return avgPA,avgPB,avganglePA



In [5]:
# def drawfaceline(nose,ear,avgPA,avgPB):
def drawfaceline(avgPA,avgPB,avganglePA):
    
    
    nose = np.array([337,157]) #have to be measurement before
    ear = np.array([196,230]) #have to be measurement before
#     path = './Face_Portrait/val/fusion_1.jpg'
#     path = '../face_side_val/img5.jpg'
    path = '../face_side_val/face_side.jpg'    
    img = cv2.imread(path)
#     img = cv2.resize(img, (640,480), interpolation=cv2.INTER_AREA)
    img = cv2.resize(img, (400,400), interpolation=cv2.INTER_AREA)

    cv2.line(img, tuple(nose), tuple(ear), (0, 0, 255), 3)
    
    X = eulard(nose,ear)
    near = (ear-nose)*avgPA
    
    if near[0] <= 0:
        avganglePA = -avganglePA
    p = (near[0]*np.cos(avganglePA) - near[1]*np.sin(avganglePA) , near[0]*np.sin(avganglePA) + near[1]*np.cos(avganglePA))
    p = p + nose
    p = p.astype(int)
    print(p) # the position is global position
    cv2.line(img, tuple(nose), tuple(p), (0, 255, 0), 2)    
    
    

    cv2.imshow('windows',img)
    
    
#     key = cv2.waitKey(0) & 0xFF
#     if key == ord('q'):
#         cv2.destroyAllWindows()
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    

In [6]:
def transform(point):
    new_point = np.array([point[2],point[1],-point[0]])
    return new_point

def mapping_on_plate():
    original_point = np.array([[300,250,280],[300,300,300],[300,300,290]]) # mm for units 
    new_camera = np.array([-800,0,800])
    face_plate_vector = np.cross((original_point[0]-original_point[1]),(original_point[0]-original_point[2]))
    portrait_point = np.array([[500,0,800],[400,20,790],[420,10,790]])
    # this is the original detected position from newcam (nose,ear,nastropy) , so we have to do transformation
#     print('portrait',portrait_point)
    
    new_portrait_point = np.empty([3,3])
#     print(new_portrait_point)
    for i,point in enumerate (portrait_point):
        new_portrait_point[i] = transform(point) + new_camera
        print(i,point)
    print(new_portrait_point)
    print('face' , face_plate_vector)
#     print(eulard(face_plate_vector,0))
    map_point = new_portrait_point + face_plate_vector * ( (np.dot(face_plate_vector,new_portrait_point) + np.dot(face_plate_vector,original_point))/ (eulard(face_plate_vector,0) * eulard(face_plate_vector,0)))
    print(map_point)
    
    return face_plate_vector, original_point , map_point, new_portrait_point

In [7]:
face_plate_vector, original_point , map_point, new_potrait_point = mapping_on_plate()
# draw3D(face_plate_vector, original_point , map_point, new_potrait_point)

0 [500   0 800]
1 [400  20 790]
2 [420  10 790]
[[  0.   0. 300.]
 [-10.  20. 400.]
 [-10.  10. 380.]]
face [-500    0    0]
[[300.   0. 300.]
 [290.  20. 400.]
 [290.  10. 380.]]


In [8]:
def draw3D(face_plate_vector, original_point , map_point, new_potrait_point):
# 建立畫布
    fig = plt.figure(figsize=(12, 8),
         facecolor='lightyellow'
        )
    # 建立 3D 座標系
    ax = fig.gca(fc='whitesmoke',
        projection='3d' 
       )# 二元函數定義域平面
    x = np.linspace(0, 9, 9)
    y = np.linspace(0, 9, 9)
    X, Y = np.meshgrid(x, y)
    # -------------------------------- 繪製 3D 圖形 --------------------------------
    # 平面 z=4.5 的部分
    ax.plot_surface(X,
        Y,
        Z=X*0+4.5,
        color='g',
        alpha=0.6
        ) 
     # 平面 y=4.5 的部分
    ax.plot_surface(X,
        Y=X*0+4.5,
        Z=Y,
        color='g',
        alpha=0.6
        ) 
    # 平面 x=4.5 的部分
    ax.plot_surface(X=X*0+4.5,
        Y=Y,
        Z=X, 
        color='g',
        alpha=0.6
        ) 
    
#     add face_plate
    dot = np.dot(face_plate_vector,original_point)
    ax.plot_surface(X = X,
                    Y = Y, 
                    Z = (dot-face_plate_vector[0] *X-face_plate_vector[1] *Y)/face_plate_vector[2],
                           color='r',
                    alpha=0.6) 
    # -------------------------------- --------------------------------
    # 設定座標軸標題和刻度
    ax.set(xlabel='X',
      ylabel='Y',
      zlabel='Z',
      xlim=(0, 9),
      ylim=(0, 9),
      zlim=(0, 9),
      xticks=np.arange(0, 10, 2),
      yticks=np.arange(0, 10, 1),
      zticks=np.arange(0, 10, 1)
      )
    # 調整視角
    ax.view_init(elev=15, # 仰角
        azim=60 # 方位角
       )

    # 顯示圖形
    plt.show()

In [ ]:
if __name__ == "__main__":
    data = {}
    data["pointsall"] = []
    
#Read the destination image
    for i in range(1,11): # last number will not count
        img = '../Face_Portrait_dataset/out/fusion_' + str(i) + '.jpg'
        img_dst = cv2.imread(img)

        # print("Click on the screen and press any key for end process")
        
        
#         each image needs to fetch three point , nostril,earhole,nastronyx
        points  = get_points(img_dst)

        print("\npoints list of fusion_" + str(i) +':')
        # pointsall = pointsall.extend(points)
        data["pointsall"].append(points)
        print(points)
        
    print("\npointsall")
    print(data["pointsall"])
    avgPA,avgPB,avganglePA = caculation(data["pointsall"])


In [ ]:
drawfaceline(avgPA,avgPB,avganglePA)

[248 236]


In [11]:
avgPA,avgPB,avganglePA = caculation(data["pointsall"])

TypeError: string indices must be integers

In [ ]:
# n2e_parameters saving

save_path = 'predicted_n2e_para_0925.txt'

f = open(save_path, 'w')

f.write('avgPA :\n')
f.write(str(avgPA))
f.write('\navgPB :\n')
f.write(str(avgPB))
f.write('\navganglePA :\n')
f.write(str(avganglePA))

f.close()

In [10]:
# n2e_parameter loading in

save_path = 'predicted_n2e_para_0925.txt'

with open(save_path, 'r') as f:
    data = f.read()
    print(data)
#     print('------------------')

    newtext = data.split("\n")
    print(newtext)
    
    avgPA = float(newtext[1])
    avgPB = float(newtext[3])
    avganglePA = float(newtext[5])


    
    f.close()

avgPA :
0.7517640816245745
avgPB :
0.33474129464681013
avganglePA :
0.25400983105467884
['avgPA :', '0.7517640816245745', 'avgPB :', '0.33474129464681013', 'avganglePA :', '0.25400983105467884']


In [ ]:
print(type(avgPA))
print(avgPA)